In [35]:
### 통합 summgnn 포맷 데이터셋 생성
import random

random.seed(42)

input_files = ["../drugbank/train.txt", "../drugbank/dev.txt", "../drugbank/test.txt"]
output_file = "../drugbank/drugbank.txt"

lines = []

# Read
for input_file in input_files:
    with open(input_file, "r") as file:
        lines.extend(file.readlines())

random.shuffle(lines)

# Write
with open(output_file, "w") as file:
    file.writelines(lines)


## Save only Gene-Compound Relation

In [22]:
import pandas as pd
import numpy as np

## hetionet
# data/hetionet/hetionet-v1.0-edges.sif
# data/hetionet/hetionet-v1.0-nodes.tsv
file_path = "../hetionet/hetionet-v1.0-edges.sif"
hetio = pd.read_csv(file_path) # , sep='\t'
print(hetio)

                                  source\tmetaedge\ttarget
0         Gene::9021\tGpBP\tBiological Process::GO:0071357
1        Gene::51676\tGpBP\tBiological Process::GO:0098780
2           Gene::19\tGpBP\tBiological Process::GO:0055088
3         Gene::3176\tGpBP\tBiological Process::GO:0010243
4         Gene::3039\tGpBP\tBiological Process::GO:0006898
...                                                    ...
2250192          Anatomy::UBERON:0000057\tAeG\tGene::65009
2250193          Anatomy::UBERON:0000474\tAeG\tGene::80279
2250194           Anatomy::UBERON:0002048\tAeG\tGene::1211
2250195           Anatomy::UBERON:0002048\tAeG\tGene::8843
2250196           Anatomy::UBERON:0002240\tAeG\tGene::2124

[2250197 rows x 1 columns]


In [23]:
new = pd.DataFrame()
for i in range(len(hetio)):
    rel = hetio.iat[i,0]
    if "Gene" in rel and "Compound" in rel:
        print(rel)
        new = new.append(pd.Series(rel), ignore_index=True)
print(len(new))

### save result : only undirected compound - gene  compound - gene 만 따로 저장 
new.to_csv("../hetionet/compoundgene_hetio_edges.sif", header=["source\tmetaedge\ttarget"], index=False) #  sep='\t',

Compound::DB00514	CbG	Gene::1136
Compound::DB00686	CbG	Gene::2246
Compound::DB00786	CbG	Gene::4317
Compound::DB01209	CbG	Gene::4988
Compound::DB01588	CbG	Gene::2560
Compound::DB00175	CbG	Gene::1577
Compound::DB00813	CbG	Gene::100861540
Compound::DB00869	CbG	Gene::765
Compound::DB00172	CbG	Gene::11212
Compound::DB01250	CbG	Gene::240
Compound::DB00313	CbG	Gene::5742
Compound::DB00217	CbG	Gene::3758
Compound::DB06262	CbG	Gene::6530
Compound::DB01115	CbG	Gene::8912
Compound::DB00213	CbG	Gene::1544
Compound::DB00203	CbG	Gene::1576
Compound::DB00150	CbG	Gene::10352
Compound::DB00909	CbG	Gene::6328
Compound::DB00847	CbG	Gene::4887
Compound::DB00451	CbG	Gene::353189
Compound::DB00575	CbG	Gene::148
Compound::DB01218	CbG	Gene::1558
Compound::DB06716	CbG	Gene::213
Compound::DB01364	CbG	Gene::6571
Compound::DB01559	CbG	Gene::2561
Compound::DB00502	CbG	Gene::1815
Compound::DB00715	CbG	Gene::1132
Compound::DB01133	CbG	Gene::5770
Compound::DB00390	CbG	Gene::480
Compound::DB01086	CbG	Gene::6332
Compou

## Excluding input datset

In [2]:
###############################################################
###  Boilerplating for removal edges from hetionet
###  -> prepare compound-gene hetio edges

import json
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

new = pd.read_csv("../hetionet/compoundgene_hetio_edges.sif", sep="\t")

# Reorder the columns
new_column_order = ['target', 'metaedge', 'source']
new = new.reindex(columns=new_column_order)

# metaedge value counts
print(new['metaedge'].value_counts())

# Detach "Compound::"
for col in ['source']:
    for i in range(len(new[col])):
        new[col][i] = new[col][i].split("::")[1]
# print(new)

CdG    21102
CuG    18756
CbG    11571
Name: metaedge, dtype: int64


### module for excluding sumgnn input format

In [3]:
## excluding_dataset_dir : sumgnn format input datset의 상대경로
## 
excluding_dataset_dir = "../vec/dti2vec.txt"


datasetname = excluding_dataset_dir.split(".")[-2].split("/")[-1] ## "dti2vec", "kiba", "davis" ...
print(f"Input Dataset : {datasetname}")
dataset = pd.read_csv(excluding_dataset_dir, sep=' ', header=None)
## Only positive samples
dataset = dataset[dataset[2] == 1].reset_index()
# print(dataset)

## Gene decoding: sumgnn decoding to hetionet ID
with open('entity_drug.json', 'r') as f:
    data = json.load(f)
decoding1 = []
data_inverse = {v: k for k, v in data.items()}  ## Create inverse dictionary
for j in range(len(dataset[0])):
    gene_id = dataset[0][j]
    if gene_id in data_inverse:
        decoding1.append(data_inverse[gene_id])
dataset['Gene_ID'] = decoding1

## Drug decoidng: sumgnn decoding to hetionet ID
with open('node2id.json') as f:
    data = json.load(f)
decoding2 = []
data_inverse = {v: k for k, v in data.items()}  ## Create inverse dictionary
for j in range(len(dataset[1])):
    drug_id = dataset[1][j]
    if drug_id in data_inverse:
        decoding2.append(data_inverse[drug_id])
dataset['Drug_ID'] = decoding2
# print(dataset)
# print(new)

## including in Both of hetionet and input dataset
matching_samples = new[new[['target', 'source']].apply(tuple, axis=1).isin(dataset[['Gene_ID', 'Drug_ID']].apply(tuple, axis=1))]
# print(matching_samples)

## Attach "Compound::"
matching_samples['source'] = "Compound::"+ matching_samples['source']

## Restore the column order
column_order = ['source', 'metaedge', 'target']
matching_samples = matching_samples.reindex(columns=column_order)

## Origin hetio edges 
file_path = "../hetionet/hetionet-v1.0-edges.sif"
hetio = pd.read_csv(file_path, sep='\t') # 
# print(hetio)

## hetio edge 원본과 중복되는 샘플을 제외
matching = hetio[hetio[['source', 'target']].apply(tuple, axis=1).isin(matching_samples[['source', 'target']].apply(tuple, axis=1))]
filtered_hetio = hetio[~hetio.index.isin(matching.index)]
filtered_hetio = filtered_hetio.reset_index()
# print(filtered_hetio)

## Save
file_path = f"../hetionet/hetionet-v1.0-edges-excluding_{datasetname}.sif"
filtered_hetio.to_csv(file_path, sep='\t')
before, after = len(hetio['metaedge']), len(filtered_hetio['metaedge'])
print(f'Save dir : {file_path}')
print(f'{before} -> {after}')
print(f"# of removals : {len(hetio['metaedge'])-len(filtered_hetio['metaedge'])} ")

Input Dataset : dti2vec
Save dir : ../hetionet/hetionet-v1.0-edges-excluding_dti2vec.sif
2250197 -> 2241792
# of removals : 8405 


In [4]:
### double-check
d = "DB00445"
t = "Gene::1105"

for idx, row in new.iterrows():
    if row['source'] == d and row['target']== t:
        f1 = True
        print(row['source'], row['target'])


for idx, row in dataset.iterrows():
    if row['Drug_ID'] == d and row['Gene_ID']== t:
        f2 = True
        print(row['Drug_ID'], row['Gene_ID'])

if f1 == f2 == True:
    print("중복되는 edge")

DB00445 Gene::1105
DB00445 Gene::1105
DB00445 Gene::1105
중복되는 edge


## Generate relation2hop.txt

In [5]:
filtered_hetio

,index,source,metaedge,target
0,0,Gene::9021,GpBP,Biological Process::GO:0071357
1,1,Gene::51676,GpBP,Biological Process::GO:0098780
2,2,Gene::19,GpBP,Biological Process::GO:0055088
3,3,Gene::3176,GpBP,Biological Process::GO:0010243
4,4,Gene::3039,GpBP,Biological Process::GO:0006898
...,...,...,...,...
2241787,2250192,Anatomy::UBERON:0000057,AeG,Gene::65009
2241788,2250193,Anatomy::UBERON:0000474,AeG,Gene::80279
2241789,2250194,Anatomy::UBERON:0002048,AeG,Gene::1211
2241790,2250195,Anatomy::UBERON:0002048,AeG,Gene::8843


In [6]:
with open('entity_drug.json', 'r') as f:
    data = json.load(f)
print(data['Gene::801'])
with open("metaedge_encoding.json", 'r') as f:
    data2 = json.load(f)
print(data2)

1710
{'GiG': 0, 'CrC': 1, 'DdG': 2, 'DpS': 3, 'DlA': 4, 'CtD': 5, 'CbG': 6, 'CuG': 7, 'DaG': 9, 'AdG': 11, 'CpD': 10, 'AeG': 12, 'GcG': 13, 'GpMF': 14, 'PCiC': 15, 'GpCC': 16, 'Gr>G': 17, 'DuG': 19, 'CcSE': 21}


In [7]:
filtered_hetio['source_encoding'] = [data.get(key) for key in filtered_hetio['source']]
filtered_hetio['edge_encoding'] = [data2.get(key) for key in filtered_hetio['metaedge']]
filtered_hetio['target_encoding'] = [data.get(key) for key in filtered_hetio['target']]
tmp = filtered_hetio.dropna()
# print(tmp)
import math
tmp['source_encoding'] = tmp['source_encoding'].apply(lambda x: int(float(x)) if not math.isnan(x) else None)
tmp['edge_encoding'] = tmp['edge_encoding'].apply(lambda x: int(float(x)) if not math.isnan(x) else None)
tmp['target_encoding'] = tmp['target_encoding'].apply(lambda x: int(float(x)) if not math.isnan(x) else None)

In [8]:
print(tmp['source_encoding'].dtype)
tmp['edge_encoding'].value_counts()
tmp

int64


,index,source,metaedge,target,source_encoding,edge_encoding,target_encoding
559504,559504,Gene::801,GiG,Gene::7428,1710,0,1711
559505,559505,Gene::5987,GiG,Gene::9412,1712,0,1713
559506,559506,Gene::5747,GiG,Gene::79738,1714,0,1715
559507,559507,Gene::3725,GiG,Gene::10514,1716,0,1717
559508,559508,Gene::10014,GiG,Gene::55844,1718,0,1719
...,...,...,...,...,...,...,...
2241787,2250192,Anatomy::UBERON:0000057,AeG,Gene::65009,18542,12,10793
2241788,2250193,Anatomy::UBERON:0000474,AeG,Gene::80279,18626,12,3535
2241789,2250194,Anatomy::UBERON:0002048,AeG,Gene::1211,18610,12,7276
2241790,2250195,Anatomy::UBERON:0002048,AeG,Gene::8843,18610,12,18919


In [9]:
tmp['metaedge'].value_counts()

AeG     526407
Gr>G    265672
GiG     147164
AdG     102240
GpMF     97222
GpCC     73566
GcG      61690
DaG      12623
DuG       7731
DdG       7623
CcSE      4223
DlA       3602
DpS       3357
CbG        938
CuG        340
CrC        170
PCiC        47
CtD         12
CpD          6
Name: metaedge, dtype: int64

In [10]:
for idx, row in tmp.iterrows():
    if row['metaedge'] == 'CpD':
        print(row)

index                          772104
source              Compound::DB00707
metaedge                          CpD
target             Disease::DOID:5041
source_encoding                 18830
edge_encoding                      10
target_encoding                 17975
Name: 763712, dtype: object
index                          772123
source              Compound::DB00286
metaedge                          CpD
target             Disease::DOID:1612
source_encoding                 16956
edge_encoding                      10
target_encoding                 17039
Name: 763731, dtype: object
index                          772143
source              Compound::DB01398
metaedge                          CpD
target             Disease::DOID:8893
source_encoding                 16920
edge_encoding                      10
target_encoding                 17023
Name: 763751, dtype: object
index                           772197
source               Compound::DB00600
metaedge                           CpD
t

## Save to relations_2hop.txt

In [ ]:
tmp[['source_encoding', 'target_encoding', 'edge_encoding']].to_csv('../vec/relations_2hop.txt', sep=' ', index=False, header=False)